In [1]:
from pathlib import Path
from typing import List
import traceback
from nncf import compress_weights
from tqdm import trange
import json
import shutil
import random
import copy
from optimum.intel.openvino import OVModelForCausalLM
from transformers import (
    AutoTokenizer,
    AutoConfig
)
from nncf.parameters import CompressWeightsMode
from whowhatbench import Evaluator

from optimum.utils import NormalizedTextConfig, NormalizedConfigManager
from optimum.exporters import TasksManager
from openvino import Core
core = Core()

TasksManager._SUPPORTED_MODEL_TYPE["stablelm-epoch"] = TasksManager._SUPPORTED_MODEL_TYPE["llama"]
NormalizedConfigManager._conf["stablelm-epoch"] = NormalizedTextConfig.with_args(
    num_layers="num_hidden_layers",
    num_attention_heads="num_attention_heads",
)

ROOT_DIR = Path('/home/devuser/nlyalyus/projects/lm-evaluation-harness/cache/stable-zephyr-3b-dpo')
gold_folder = ROOT_DIR / "fp16"
gold_ir_path = gold_folder / "openvino_model.xml"
config = AutoConfig.from_pretrained(gold_folder, trust_remote_code=True)
model_gold = OVModelForCausalLM.from_pretrained(gold_folder, config=config, trust_remote_code=True, use_cache=True)
tokenizer_gold = AutoTokenizer.from_pretrained(gold_folder)
evaluator = Evaluator(base_model=model_gold, tokenizer=tokenizer_gold)#, gt_data='gold.csv')
evaluator.dump_gt('gold_all.csv')

# try:
#     num_internal_ids = 5 #229 - 2
#     list_scores_per_stage: List[List[int]] = []
#     optimal_ids = []
#     for stage in range(num_internal_ids):
#         metric_per_id = []
#         for i in trange(num_internal_ids, desc='Exhaustive search for sensitive layer'):
#             if i in optimal_ids:
#                 metric_per_id.append(0)
#                 continue
#             # gold_ov_model = core.read_model(model=gold_ir_path)
#             force_int8_ids = optimal_ids + [i]
#             print('evaluate config: ', force_int8_ids)
#             compressed_model = compress_weights(gold_ov_model, mode=CompressWeightsMode.INT4_SYM, ratio=1, group_size=64, force_int8_ids=force_int8_ids)
#             cmp_model = OVModelForCausalLM(compressed_model, config=config, trust_remote_code=True, use_cache=True, model_save_dir=gold_folder)
#             all_metrics_per_question, all_metrics = evaluator.score(cmp_model)
#             print(all_metrics)
#             similarity = float(all_metrics['similarity'])
#             sdt_norm = float(all_metrics['SDT norm'])
#             score = (similarity + sdt_norm) / 2

#             # print(all_metrics_per_question)
#             metric_per_id.append(score)
#         best_score = max(metric_per_id)
#         optimal_id = metric_per_id.index(best_score)
#         print('all scores=', metric_per_id)
#         print('best score', best_score)
#         print('choose layer with index=', optimal_id)
#         optimal_ids.append(optimal_id)
#         list_scores_per_stage.append(metric_per_id)
# except Exception as error:
#     print(traceback.print_exc())
# finally:
#     model_cache_dir = gold_folder / 'model_cache'
#     if model_cache_dir.exists():
#         shutil.rmtree(model_cache_dir)

#     scores_path = gold_folder / 'greedy_search_result.json'
#     results = {
#         'scores_per_stage': list_scores_per_stage,
#         'optimal_ids': optimal_ids,
#     }
#     with scores_path.open('w') as f:
#         json.dump(results, f, indent=2)

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to CPU ...
Setting OpenVINO CACHE_DIR to /home/devuser/nlyalyus/projects/lm-evaluation-harness/cache/stable-zephyr-3b-dpo/fp16/model_cache
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_i

In [6]:
from pathlib import Path
from optimum.intel.openvino import OVModelForCausalLM
from transformers import AutoConfig
from openvino import Core

ROOT_DIR = Path('/home/devuser/nlyalyus/projects/lm-evaluation-harness/cache/stable-zephyr-3b-dpo')
gold_path = ROOT_DIR / "fp16"
ir_path =  gold_path / "openvino_model.xml"
config_gold = AutoConfig.from_pretrained(gold_path, trust_remote_code=True)

core = Core()
ov_model = core.read_model(model=ir_path)

from optimum.utils import NormalizedTextConfig, NormalizedConfigManager
from optimum.exporters import TasksManager

TasksManager._SUPPORTED_MODEL_TYPE["stablelm-epoch"] = TasksManager._SUPPORTED_MODEL_TYPE["llama"]
NormalizedConfigManager._conf["stablelm-epoch"] = NormalizedTextConfig.with_args(
    num_layers="num_hidden_layers",
    num_attention_heads="num_attention_heads",
)

model_gold = OVModelForCausalLM(ov_model, config=config_gold, trust_remote_code=True, use_cache=True, model_save_dir=gold_path)

Compiling the model to CPU ...
Setting OpenVINO CACHE_DIR to /home/devuser/nlyalyus/projects/lm-evaluation-harness/cache/stable-zephyr-3b-dpo/fp16/model_cache


In [5]:
import os
from pathlib import Path
from typing import List
import traceback
from nncf import compress_weights
from tqdm import trange
import json
import shutil
import random
import copy
from optimum.intel.openvino import OVModelForCausalLM
from transformers import (
    AutoTokenizer,
    AutoConfig
)
from contextlib import redirect_stdout, redirect_stderr
from nncf.parameters import CompressWeightsMode
from whowhatbench import Evaluator

from optimum.utils import NormalizedTextConfig, NormalizedConfigManager
from optimum.exporters import TasksManager
from openvino import Core
core = Core()

TasksManager._SUPPORTED_MODEL_TYPE["stablelm-epoch"] = TasksManager._SUPPORTED_MODEL_TYPE["llama"]
NormalizedConfigManager._conf["stablelm-epoch"] = NormalizedTextConfig.with_args(
    num_layers="num_hidden_layers",
    num_attention_heads="num_attention_heads",
)
#MODEL_NAME = 'stable-zephyr-3b-dpo'
MODEL_NAME = 'llama-2-7b-chat-hf'

cache_dir = Path(os.readlink('/home/devuser/nlyalyus/projects/lm-evaluation-harness/cache'))
ROOT_DIR = cache_dir / MODEL_NAME
gold_folder = ROOT_DIR / "fp16"
gold_ir_path = gold_folder / "openvino_model.xml"
gold_csv = gold_folder / 'gold_all.csv'
config = AutoConfig.from_pretrained(gold_folder, trust_remote_code=True)
tokenizer_gold = AutoTokenizer.from_pretrained(gold_folder)
print('gold path:', gold_csv.resolve())
if gold_csv.exists():
    evaluator = Evaluator(tokenizer=tokenizer_gold, gt_data=gold_csv, test_data_path=gold_csv)
else:
    model_gold = OVModelForCausalLM.from_pretrained(gold_folder, config=config, trust_remote_code=True, use_cache=True)
    evaluator = Evaluator(base_model=model_gold, tokenizer=tokenizer_gold)
    evaluator.dump_gt(gold_csv)

EXP_NAMES = [
    # 'int4_g64_nozp_r80_greedy0_anti',
    # 'int4_g64_nozp_r80_greedy0',
    # 'int4_g64_nozp_r80_greedy1',
    'int4_g64_nozp_r80',
    'int4_g64_nozp_r80_hawq_in',
    'int4_g64_nozp_r80_max_var',
    'int4_g64_nozp_r80_mean_var',
    'int4_g64_nozp_r80_mean_max',
    # 'int4_g64_nozp_r80_criteria_OUT2',
    # 'int4_g64_nozp_r80_baseline',
    # 'int4_g64_nozp_r80_mean_var',
    # 'int4_g64_nozp_r80_max_var',
    # 'int4_g64_nozp_r80_mean_max',
]

for exp_name in EXP_NAMES:
    cmp_ir_path = ROOT_DIR / exp_name
    cmp_model = OVModelForCausalLM.from_pretrained(cmp_ir_path, config=config, trust_remote_code=True, use_cache=True)
    all_metrics_per_question, all_metrics = evaluator.score(cmp_model)
    print(all_metrics)
    similarity = float(all_metrics['similarity'].iloc[0])
    sdt_norm = float(all_metrics['SDT norm'].iloc[0])
    score = (similarity + 1 - sdt_norm) / 2
    # print(all_metrics_per_question)
    print('final score=', score)
    model_cache_dir = cmp_ir_path / 'model_cache'
    if model_cache_dir.exists():
        shutil.rmtree(model_cache_dir)
    all_metrics.to_csv(cmp_ir_path / 'eval.csv')

gold path: /dev/data/nlyalyus/cache/llama-2-7b-chat-hf/fp16/gold_all.csv


The argument `trust_remote_code` is to be used along with export=True. It will be ignored.


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/dev/data/nlyalyus/cache/llama-2-7b-chat-hf/int4_g64_nozp_r80'. Use `repo_type` argument if needed.

In [27]:
s = '/home/devuser/nlyalyus/projects/lm-evaluation-harness/cache/stable-zephyr-3b-dpo/fp16/greedy_search_result.json'
with open(s) as f:
    j = json.load(f)
similarity = j['scores_per_stage'][1]
indexes_of_layers_in_ascending_order_of_errors = [
    i[0] for i in sorted(enumerate(similarity), reverse=True, key=lambda x: x[1])
]
optimal_ids_1 = indexes_of_layers_in_ascending_order_of_errors[:19]
[similarity[i] for i in optimal_ids_1]

[0.7065786778926849,
 0.6889446919614619,
 0.6857339669828829,
 0.6849753201007842,
 0.6849614435718172,
 0.6837329502855793,
 0.68353123857129,
 0.68353123857129,
 0.6833621461545267,
 0.6833621461545267,
 0.6833621461545267,
 0.6833621461545267,
 0.6833621461545267,
 0.6833621461545267,
 0.6831865429878234,
 0.6825421620298315,
 0.6820468101127088,
 0.6818355639134683,
 0.6816900074481964]

In [18]:
optimal_ids_0

[96,
 29,
 30,
 77,
 41,
 156,
 88,
 62,
 53,
 154,
 84,
 21,
 22,
 7,
 19,
 93,
 12,
 28,
 138,
 223]

In [28]:
optimal_ids_1

[28,
 115,
 41,
 84,
 165,
 48,
 36,
 135,
 2,
 8,
 15,
 51,
 64,
 199,
 134,
 29,
 21,
 106,
 22]

In [36]:
set(optimal_ids_1).difference(set(optimal_ids))

{2, 8, 15, 36, 48, 51, 64, 106, 115, 134, 135, 165, 199}

In [37]:
set(optimal_ids_1).intersection(set(optimal_ids))


{21, 22, 28, 29, 41, 84}

In [15]:
s1= '/dev/data/nlyalyus/cache/llama-2-7b-chat-hf/int4_sym_g128_r80_mean_var/eval.csv'
s2= '/dev/data/nlyalyus/cache/llama-2-7b-chat-hf/int4_sym_g128_r80_max_var/eval.csv'
from pathlib import Path
n1 = Path(s1).parent.name
n2 = Path(s2).parent.name
import pandas as pd
p1=pd.read_csv(s1)
p2=pd.read_csv(s2)
p2['metric'] = [n1]
p1['metric'] = [n2]
pd.concat([p2, p1], axis=0)

,Unnamed: 0,similarity,FDT,SDT,FDT norm,SDT norm,metric
0,0,0.920754,27.368421,62.157895,0.21121,0.478138,int4_sym_g128_r80_mean_var
0,0,0.904372,19.947368,65.736842,0.15394,0.505944,int4_sym_g128_r80_max_var


In [35]:
if 'Weighted' not in p1.columns:
    p1['Weighted'] = (p1['similarity'] + 1 - p1['SDT norm']) / 2
    # column_names = ['model', 'exp_name', 'Weighted', 'exp_name', 'similarity', 'FDT', 'SDT', 'SDT norm', 'FDT norm']
    column_names = ['metric', 'Weighted', 'similarity', 'FDT', 'SDT', 'SDT norm', 'FDT norm']
    p1[column_names]

,metric,Weighted,similarity,FDT,SDT,SDT norm,FDT norm
0,int4_sym_g128_r80_max_var,0.699214,0.904372,19.947368,65.736842,0.505944,0.15394
